In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PIL
from tensorflow.keras import datasets, layers, models
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D,Activation, Flatten, Dropout, Dense
from keras_tuner import HyperModel, HyperParameters, RandomSearch
import keras_tuner as kt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ModelCheckpoint

In [2]:
from PIL import Image
import os

# create data generators

In [3]:
target_size = (96,96)

In [4]:
batch_size = 128

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range = .3,
    zoom_range = .3,
    horizontal_flip=True,
    brightness_range = [.4,1.5],
    rotation_range = 30,
    width_shift_range=0.15,
    height_shift_range=0.15
    #featurewise_std_normalization=False # not sure how this will work here
)

In [6]:
train_generator = train_datagen.flow_from_directory(
        r'C:/Users/natha/OneDrive/Desktop/datascience/machine learning and predictive/final/dataset/output/train',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 32938 images belonging to 151 classes.


In [7]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [8]:
test_data_generator = test_datagen.flow_from_directory(
    r'C:/Users/natha/OneDrive/Desktop/datascience/machine learning and predictive/final/dataset/output/val',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 8314 images belonging to 151 classes.


# tune hyper parameters

In [9]:
#,validation_data = test_data_generator

In [20]:
model = keras.Sequential()
model.add(Conv2D(32, 3, padding = 'same', activation = 'relu', input_shape =(96, 96, 3), kernel_initializer = 'he_normal'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(151, activation = 'softmax'))

In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate = 1e-2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 96, 96, 32)        896       
                                                                 
 batch_normalization_9 (Batc  (None, 96, 96, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 48, 48, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 batch_normalization_10 (Bat  (None, 48, 48, 64)      

In [9]:
checkpoint = ModelCheckpoint('advanced96/best_model.h5', verbose = 1, monitor = 'val_accuracy', save_best_only = True)

In [10]:
model = keras.models.load_model('advanced96/model_advanced96.h5')

In [ ]:
history = model.fit(train_generator, epochs = 30, validation_data = test_data_generator,
                              callbacks = [checkpoint])

Epoch 1/30
258/258 [==============================] - ETA: 0s - loss: 1.9416 - accuracy: 0.5039
Epoch 1: val_accuracy improved from 0.53777 to 0.56351, saving model to advanced96\best_model.h5
258/258 [==============================] - 1124s 4s/step - loss: 1.9416 - accuracy: 0.5039 - val_loss: 1.7432 - val_accuracy: 0.5635
Epoch 2/30
258/258 [==============================] - ETA: 0s - loss: 1.9142 - accuracy: 0.5118
Epoch 2: val_accuracy improved from 0.56351 to 0.56856, saving model to advanced96\best_model.h5
258/258 [==============================] - 1133s 4s/step - loss: 1.9142 - accuracy: 0.5118 - val_loss: 1.7370 - val_accuracy: 0.5686
Epoch 3/30
258/258 [==============================] - ETA: 0s - loss: 1.8746 - accuracy: 0.5180
Epoch 3: val_accuracy did not improve from 0.56856
258/258 [==============================] - 1094s 4s/step - loss: 1.8746 - accuracy: 0.5180 - val_loss: 1.9380 - val_accuracy: 0.5244
Epoch 4/30
258/258 [==============================] - ETA: 0s - loss

In [ ]:
# Plot learning curves
fig = plt.figure(figsize = (17, 4))
    
plt.subplot(121)
plt.plot(history.history['accuracy'], label = 'acc')
plt.plot(history.history['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid()
plt.title(f'accuracy')

plt.subplot(122)
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.legend()
plt.grid()
plt.title(f'loss')

In [ ]:
hisorydf = pd.DataFrame(history.history)
hisorydf.to_csv('advanced96/my_model_advanced96_60.csv')

In [ ]:
model.save('advanced96/model_advanced96.h5')

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('whothatpokemon/pikachu.png', target_size=target_size)

#preprocess the image
my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)

#make the prediction
prediction = model.predict(my_image)

In [ ]:
class_labels = []
for i in os.listdir('output/train'):
    class_labels.append(i)

In [ ]:
predicted_class_index = np.argmax(prediction)
predicted_class_label = f'Predicted: {class_labels[predicted_class_index]} {round(prediction[0][predicted_class_index] * 100, 2)}%'

print(f"The predicted class is: {predicted_class_label}")